In [1]:
2

2

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../loaders')

In [ ]:
from raft import RAFTExhaustiveDataset
from config import config_parser
import util

In [ ]:
import util
import imageio
import os
import json
import cv2

def save_video(frames, out_path):
    """
        Args:
            frames: list of np.ndarray of shape (H, W, 3) and dtype np.uint8
    """
    try:
        imageio.mimwrite(out_path, frames, quality=8, fps=10)

        print(f'💾 Saved video to {out_path}')
    except Exception as e:
        print(f'💾⚠️ Failed to save video to {out_path}')
        print(e)
        print(f'frames[0].shape: {frames[0].shape}, frames[0].dtype: {frames[0].dtype}')

def save_flow_as_video(out_path, flows):
    flow_imgs = [visualize_flow_as_arrows(flow_map) for flow_map in tqdm(flows, "converting flows to images")]

    save_video(flow_imgs, out_path)

    return flow_imgs

def count_map_to_image(count_maps, i):

    # expand pixel value into 3 dimensions
    count_map = np.repeat(count_maps[i][:, :, np.newaxis], 3, axis=2)

    # normalize across all count maps
    count_map = count_map / np.max(count_maps)
    
    # convert to np.uint8
    count_map = (count_map * 255).astype(np.uint8)

    return count_map

def draw_text(frames, text, location):
    """ Returns array of frames with text drawn on them """
    for i in range(len(frames)):
        frames[i] = cv2.putText(frames[i], text, location, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return frames
def concat_frames(frames1, frames2, left_label=None, right_label=None):
    """ Returns array of side-by-side frames of shape (N, H, W*2, 3) """
    concatted = np.concatenate([frames1, frames2], axis=2)

    if left_label is not None:
        concatted = draw_text(concatted, left_label, (10, 50))

    if right_label is not None:
        concatted = draw_text(concatted, right_label, (concatted.shape[2]//2 + 10, 50))
    
    return concatted

def save_json(out_path, data):
    """ Saves data to json file """
    with open(out_path, 'w') as f:
        json.dump(data, f)
    print(f'💾 Saved json to {out_path}')
    
    

In [ ]:
import numpy as np
import imageio
from tqdm import tqdm

class PreprocessingInspector():
    def __init__(self, seq_name, data_dir):
        self.seq_name = seq_name
        self.data_dir = data_dir

        self.flow_dir = os.path.join(DATA_DIR, 'raft_exhaustive')
        self.flow_mask_dir = os.path.join(DATA_DIR, 'raft_masks')
        self.img_dir = os.path.join(DATA_DIR, 'color')
        self.img_names = sorted(os.listdir(self.img_dir))
    

    def visualize_count_maps(self, out_dir=None):
        if out_dir is None:
            out_dir = os.path.join(self.seq_name, 'count_maps')
        
        count_dir = os.path.join(DATA_DIR, 'count_maps')
        count_img_names = sorted(os.listdir(count_dir))

        count_maps = [imageio.imread(os.path.join(count_dir, img_name)) for img_name in tqdm(count_img_names, desc="Loading count maps")]
        count_images = [count_map_to_image(count_maps, i) for i in tqdm(range(len(count_maps)), desc="Converting count maps to images")]

        os.makedirs(out_dir, exist_ok=True)
        save_video(count_images, os.path.join(out_dir, 'count_maps.mp4'))
    
    def _load_flow_maps(frame_interval, load_masks=True, load_flows=True):
        flows_raw = []
        flows_masks = []

        for i in tqdm(range(0, len(self.img_names)-frame_interval), desc='Loading flow maps'):
            img_name1 = self.img_names[i]
            img_name2 = self.img_names[i+frame_interval]

            if load_flows:
                # get flow for this pair of frames
                flow_file = os.path.join(self.flow_dir, '{}_{}.npy'.format(img_name1, img_name2))
                flow = np.load(flow_file)
                flows_raw.append(flow)

            if load_masks:
                # get mask for this pair of frames
                mask_file = os.path.join(self.flow_mask_dir, '{}_{}.png'.format(img_name1, img_name2))
                masks = imageio.imread(mask_file) / 255.

                cycle_consistency_mask = masks[..., 0] > 0
                occlusion_mask = masks[..., 1] > 0

                mask = cycle_consistency_mask | occlusion_mask

                if mask.sum() == 0:
                    invalid = True
                    mask = np.ones_like(cycle_consistency_mask)
                else:
                    invalid = False
                
                flows_masks.append(mask)
        
        return flows_raw, flows_masks
    def visualize_raft_flows(self, out_dir=None, frame_interval=1):
        
        flows_raw, flows_masks = self._load_flow_maps(frame_interval)
        if out_dir is None:
            out_dir = os.path.join(self.seq_name, 'flows')

        stats = {
            'kept_correspondences': int(sum([mask.sum() for mask in flows_masks])),
            'total_correspondences': int(sum([mask.shape[0] * mask.shape[1] for mask in flows_masks])),
        }
        stats['percentage_kept'] = stats['kept_correspondences'] / stats['total_correspondences']
        print(stats)
        save_json(os.path.join(out_dir, f'stats_frame_interval={frame_interval}.json'), stats)
        
        print(f"Across the entire video, kept {stats['kept_correspondences']} out of {stats['total_correspondences']} pixel-to-pixel correspondences ({stats['percentage_kept'] * 100}%)")
        flows_filtered = [flow * mask[..., np.newaxis] for flow, mask in zip(flows_raw, flows_masks)]
        os.makedirs(out_dir, exist_ok=True)
        flows_raw_img = save_flow_as_video(os.path.join(out_dir, f'flow_unfiltered_frame_interval={frame_interval}.mp4'), flows_raw)
        flows_filtered_img = save_flow_as_video(os.path.join(out_dir, f'flow_filtered_frame_interval={frame_interval}.mp4'), flows_filtered)

        flows_side_by_side = concat_frames(np.array(flows_raw_img), np.array(flows_filtered_img), left_label='Unfiltered', right_label='Filtered')
        save_video(flows_side_by_side, os.path.join(out_dir, f'flow_side_by_side_frame_interval={frame_interval}.mp4'))

        masks_imgs = [count_map_to_image(flows_masks, i) for i in range(len(flows_masks))]
        save_video(masks_imgs, os.path.join(out_dir, f'masks_frame_interval={frame_interval}.mp4'))

    def plot_filtering_percentage_across_frame_intervals(self, frame_intervals=[1, 2, 4, 8, 16, 32]):
        results = []
        for frame_interval in tqdm(frame_intervals, desc=f"Computing stats for {len(frame_intervals)} frame intervals"):
            _, flows_masks = self._load_flow_maps(frame_interval, load_masks=True, load_flows=False)
            stats = {
                'kept_correspondences': int(sum([mask.sum() for mask in flows_masks])),
                'total_correspondences': int(sum([mask.shape[0] * mask.shape[1] for mask in flows_masks])),
            }
            stats['percentage_kept'] = stats['kept_correspondences'] / stats['total_correspondences']

            results.append({
                'frame_interval': frame_interval,
                **stats
            })


        # Sorting results by frame_interval for plotting
        results.sort(key=lambda x: x['frame_interval'])

        # Extracting values for plotting
        x_values = [result['frame_interval'] for result in results]
        y_values = [result['percentage_kept'] for result in results]

        # Plotting the bar chart
        plt.bar(x_values, y_values, color='blue')
        plt.xlabel('Frame Intervals')
        plt.ylabel('Percentage Kept (%)')
        plt.title('Filtering Percentage across Frame Intervals')
        plt.xticks(x_values)  # This sets the tick marks on x-axis based on the frame intervals
        plt.grid(axis='y')

        # Displaying the chart
        plt.tight_layout()
        plt.show()
        
            



In [ ]:
SEQ_NAME = "soccer_short"
DATA_DIR = os.path.join("../omnimotion_videos/", SEQ_NAME)

inspector = PreprocessingInspector(SEQ_NAME, DATA_DIR)

In [ ]:
inspector.visualize_raft_flows(frame_interval=10)
inspector.visualize_count_maps()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_flow_as_arrows(flow_uv, stride=10):
    """
    Expects a two-dimensional flow image of shape.

    Args:
        flow_uv (np.ndarray): Flow UV image of shape [H,W,2]
        stride (int): The step size between arrows, for both height and width.

    Displays:
        A matplotlib figure with arrows representing the flow at each grid point.
    """
    assert flow_uv.ndim == 3, 'input flow must have three dimensions'
    assert flow_uv.shape[-1] == 2, 'input flow must have shape [H,W,2]'
    
    H, W, _ = flow_uv.shape
    
    x = np.arange(0, W, stride)
    y = np.arange(0, H, stride)
    
    x, y = np.meshgrid(x, y)
    
    u = flow_uv[::stride, ::stride, 0]
    v = flow_uv[::stride, ::stride, 1]
    
    plt.figure(figsize=(10,10))
    plt.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=1, color='r')
    plt.gca().invert_yaxis()  # To align the visualization with image coordinate system
    plt.show()


In [ ]:
2